![nsdf](https://www.sci.utah.edu/~pascucci/public/NSDF-large.png)  
[National Science Data Fabric](https://nationalsciencedatafabric.org/) 

# Converting numpy data 


- `912*816*2025*float32` Each timestep is 5 GiB.
- 900 timesteps  5T

In [ ]:
import os ,sys, time, logging,shutil
from datetime import datetime
import numpy as np

t1 = time.time()
data=np.load("/mnt/data1/DemoNSDF-feb-2023/recon_combined_1_2_3_fullres.npy")
shape,dtype=data.shape,data.dtype
D,H,W=data.shape
m,M=np.min(data),np.max(data)
print(f"np.load done in {time.time()-t1} seconds \ndtype={data.dtype} \nshape={data.shape} \nc_size={W*H*D*4:,} \nm={m} M={M}")

# Show a slice along Z

In [ ]:
import matplotlib.pyplot as plt
from scipy import ndimage

def ShowData(data,cmap="viridis"):
	print("ShowImage",data.dtype,data.shape,np.min(data),np.max(data))
	plt.imshow(data, cmap=cmap)
	plt.colorbar()
	plt.show()
	
Z=200
slice=data[Z,:,:]
ShowData(slice,cmap="viridis") 

# Import OpenVisus

In [ ]:
import OpenVisus as ov
os.environ["VISUS_DISABLE_WRITE_LOCK"]="1"
logger= logging.getLogger("OpenVisus")
# ov.SetupLogger(logger, output_stdout=True) 
print("OpenVisus imported")

# Create the IDX file

In [ ]:
idx_filename="/mnt/data1/nsdf/tmp/recon_combined_1_2_3_fullres/visus.idx"

# ************ dangerous (make sure you are in a tmp directory) ************ 
import os,sys,shutil
shutil.rmtree(os.path.dirname(idx_filename), ignore_errors=True)

fields=[ov.Field("data",str(data.dtype),"row_major")]
db=ov.CreateIdx(
	url=idx_filename,
	dims=[W,H,D],
	fields=fields,
	compression="raw")

print(type(db))
print(db.getDatasetBody().toString())

# Write data to OpenVisus

In [ ]:
t1 = time.time()
db.write(data)
print(f"db.write done in {time.time() - t1} seconds")

# Compress dataset using zip

In [ ]:
t1 = time.time()
db.compressDataset(["zip"])
print(f"db.compressDataset done in {time.time()-t1} seconds")

# Show OpenVisus data

In [ ]:
slice=db.read(x=[0,W],y=[0,H],z=[Z,Z+1], num_refinements=1)[0,:,:]
ShowData(slice)

# Show coarse to fine

In [ ]:
import os,sys
for slice in db.read(x=[0,W],y=[0,H],z=[Z,Z+1], num_refinements=3):
	ShowData(slice[0,:,:])

# Get from OpenVisus server

You need to 

- add the dataset to the `/nfs/chess/nsdf01/openvisus/lib64/python3.6/site-packages/OpenVisus/visus.config`
- launch the Open Visus server


if you want to enable dynmamic dataset  you can do for example:

```
<visus> 

<Configuration> 
    <ModVisus> 
      <Dynamic enabled="true"  filename="/nfs/chess/nsdf01/openvisus/lib64/python3.6/site-packages/OpenVisus/visus.config" msec="10000" /> 
    </ModVisus>
</Configuration>

...

</visus>
```

Get credentials to access the OpenVisus server:

In [ ]:
username,password=None,None
with open("/nfs/chess/nsdf01/openvisus/.mod_visus.identity.sh","rt") as file:
    for line in file.read().split():
        if "MODVISUS_USERNAME=" in line:
            username=line[line.index("MODVISUS_USERNAME=")+len("MODVISUS_USERNAME="):].strip()
            continue
        if "MODVISUS_PASSWORD=" in line:
            password=line[line.index("MODVISUS_PASSWORD=")+len("MODVISUS_PASSWORD="):].strip()
            continue

In [ ]:
db=ov.LoadDataset(f"https://nsdf01.classe.cornell.edu/mod_visus?dataset=recon_combined_1_fullres&~auth_username={username}&~auth_password={password}")
slice=db.read(x=[0,W],y=[0,H],z=[Z,Z+1], num_refinements=1)[0,:,:]
ShowData(slice)